In [1]:
import modin.pandas as pd
import numpy as np

In [2]:
# Quickly check annotation progress.
pd.read_parquet("wforms-ann.parquet")["status"].value_counts(dropna=False)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-30 09:29:46,864	INFO worker.py:1636 -- Started a local Ray instance.


status
-1.0    11089
 1.0      359
 NaN       76
Name: count, dtype: int64

In [3]:
annos = (
    pd.read_parquet("wforms-bat.parquet")
    .join(pd.read_parquet("wforms-occ.parquet", columns=["tot"]), how="left")
    .join(pd.read_parquet("wforms-ann.parquet"), how="left")
)

# Previously attested forms and hashtags get a special status value.
annos.loc[annos["tt_att"], "status"] = +np.inf
annos.loc[annos.index.str.startswith("#"), "status"] = -np.inf

annos[annos["fst_batch"] | annos["snd_batch"]]["status"].value_counts(dropna=False)

status
 inf    14138
-1.0     7696
-inf     4619
 1.0      359
 NaN       78
Name: count, dtype: int64

In [4]:
# Normalize categories.
annos.loc[annos["category"].isna(), "category"] = None
annos["category"] = (
    annos["category"]
    .str.replace("?", "")
    .str.replace("-1", "")
    .str.replace(";", ",")
    .str.replace(" ", "")
    .str.replace("variantgrafica", "variantegrafica")
    .str.replace("enfacigrafica", "enfasigrafica")
)
annos.loc[annos["category"] == "", "category"] = None

In [5]:
# Print category usage statistics (mind the double counting from multi-category forms!)
with pd.option_context('display.max_rows', None):
    focus = annos["fst_batch"] | annos["snd_batch"]
    display(annos[focus]["category"].value_counts())
    display(annos[focus]["category"].str.split(",").explode().value_counts())
    display(annos[focus]["category"].str.split(",").dropna().map(lambda l: l[0]).value_counts())

category
forestierismo                       62
enfasigrafica                       43
univerbazione                       38
variantegrafica                     32
macedonia                           30
prestitoadattato                    30
accorciamento                       28
suffissazione                       28
alterazione                         15
regionalismo                        13
variantegrafica,accorciamento        8
transcategorizzazione                7
variantegrafica,enfasiespressiva     5
forestierismo,accorciamento          5
denominale                           5
enfasigrafica,univerbazione          5
macedonia,deacronimico               4
deonimico                            3
acronimo                             3
prefissazione                        3
prefissoide                          3
enfasiespressiva                     3
univerbazione,accorciamento          3
ridefinizione                        3
deonimico,suffissazione              3
composto        

category
forestierismo            76
enfasigrafica            60
univerbazione            53
variantegrafica          50
accorciamento            45
suffissazione            38
macedonia                38
prestitoadattato         30
alterazione              17
regionalismo             16
enfasiespressiva         11
transcategorizzazione     9
denominale                8
deacronimico              7
deonimico                 6
acronimo                  5
prefissoide               4
ridefinizione             4
prefissazione             3
suffissoide               2
composto                  2
onomatopea                2
metaforizzazione          2
deaggettivale             1
composizione              1
leetspeak                 1
accrescitivo              1
tmesi                     1
dialettismo               1
deverbale                 1
antonomasia               1
Name: count, dtype: int64

category
forestierismo            73
enfasigrafica            53
variantegrafica          47
univerbazione            46
macedonia                35
suffissazione            34
prestitoadattato         30
accorciamento            29
alterazione              17
regionalismo             15
denominale                7
transcategorizzazione     7
deonimico                 6
enfasiespressiva          5
prefissoide               4
acronimo                  3
ridefinizione             3
prefissazione             3
onomatopea                2
suffissoide               2
composto                  2
deacronimico              1
accrescitivo              1
tmesi                     1
composizione              1
dialettismo               1
Name: count, dtype: int64

In [6]:
with pd.option_context('display.max_rows', 1000):
    focus = annos["status"].isin([0,1])
    display(annos[focus].sort_values("tot", ascending=False))

,tt_att,fst_batch,snd_batch,tot,status,category,attestation,notes
wf,,,,,,,,
rdc,False,True,False,4987,1.0,acronimo,None,None
csx,False,True,False,1064,1.0,accorciamento,None,centro-sinistra
buonagiornata,False,True,False,526,1.0,univerbazione,None,None
twitteri,False,True,False,506,1.0,"deonimico,suffissazione",None,None
piddini,False,True,False,496,1.0,"suffissazione,deacronimico",None,None
tuitteri,False,True,False,393,1.0,"deonimico,suffissazione",None,None
pidioti,False,True,False,377,1.0,macedonia,None,None
yeahhhhhh,False,True,False,365,1.0,enfasigrafica,None,None
reel,False,False,True,315,1.0,forestierismo,None,None
